In [4]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [10]:
data = pd.read_csv("./data/sentence_data.csv")

In [13]:
data.columns = ['review_no', 'city', 'country', 'Review', 'sentence']
data = data[0:10000]

In [15]:
# This function gets the polarity of reviews using Vader # 
def get_polarity(sentence):
    analyser = SentimentIntensityAnalyzer()
    score = analyser.polarity_scores(sentence)
    compound = score['compound']
    return compound
    
data['polarity'] = data['sentence'].apply(get_polarity)
data

/Users/yuankanglee/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,review_no,city,country,Review,sentence,polarity
0,0,Paris,France,No Negative,Negative,-0.5719
1,1,Levallois-Perret,France,No Negative,Negative,-0.5719
2,2,Amsterdam-Zuidoost,Netherlands,really nothing,really nothing,0.0000
3,3,Vienna,Austria,Location was good Have stayed before I e 9yrs...,Location was good,0.4404
4,3,Vienna,Austria,Location was good Have stayed before I e 9yrs...,Have stayed before I e 9yrs ago shop no longer...,0.4767
...,...,...,...,...,...,...
9995,5853,London,United Kingdom,Wonderful hotel I would highly recommend Beau...,fantastic staff and facilities Fantastic locat...,0.9349
9996,5854,London,United Kingdom,Fairly quiet location Transport links close b...,Fairly quiet location Transport links close by...,0.0000
9997,5855,Kensington,United Kingdom,Location great Very clean and comfortable,Location great Very clean and comfortable,0.8923
9998,5856,Chelsea,United Kingdom,Wifi and cell connection was low,Wifi and cell connection was low,-0.2732


In [37]:
view_data = data.loc[:, "sentence":"polarity"]
pd.set_option('display.max_rows',None)
pd.set_option('display.width', None)
view_data





,sentence,polarity
0,Negative,-0.5719
1,Negative,-0.5719
2,really nothing,0.0000
3,Location was good,0.4404
4,Have stayed before I e 9yrs ago shop no longer...,0.4767
5,Negative,-0.5719
6,Breakfast options are limited,-0.2263
7,Personally i would choose a hotel closer to an...,0.4215
8,The location is great for Paddington,0.6249
9,Beds were clean,0.4019
